<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Планетарное Движение - Задача трех тел

### Examples - Astrophysics
<section class="post-meta">
By Jonas Tjemsland, Andreas Krogen, Håkon Ånes and Jon Andreas Støvneng
</section>
Last edited: March 22nd 2018 
___

Этот пример является продолжением [Планетарного движения](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/planetary_motion.ipynb), в котором обсуждались различные решатели обыкновенных дифференциальных уравнений с постоянным шагом (ODE).

![CMoore orbit](images/CMoore1993.gif) <center>Рис. 8 орбита, открытая К. Муром в 1993 году [1].</center>


Сначала мы импортируем необходимые библиотеки и задаем некоторые общие параметры рисунка.

In [ ]:
# Import libraries
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, Image
import time
%matplotlib inline

In [ ]:
# Set common figure parameters
newparams = {'figure.figsize': (14, 7), 'axes.grid': False,
             'lines.markersize': 6, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral'}
plt.rcParams.update(newparams)

### Уравнения движения

Гравитационное притяжение между двумя объектами находится в классических условиях, задается законом тяготения Ньютона,

$$
\vec{F}_{21}(t)=\vec{F}_{12}(t) = -\frac{Gm_1m_2}{{\vec{r_{12}}(t)}^3}\vec r_{12},
$$

где $m_1$ и $m_2$ - массы двух объектов, $r$ - расстояние между ними и $G\approx 6,67\cdot на 10^{-11} \,\text{m}^3/ \text{kg}\,\text{s}^2$ - гравитационная постоянная. Закон Ньютона справедлив, если предположить, что массы объектов изотропно распределены.

Рассмотрим задачу о трех телах с тремя объектами, заданными массами $m_1$, $m_2$ и $m_3$. По принципу суперпозиции гравитационное притяжение объекта будет суммой гравитационного притяжения всех других объектов $F_1 = F_{12} + F_{13}$. Таким образом, уравнения движения (EoM) задаются

$$
\ddot{\vec{r}}_1(t) = -\frac{Gm_2}{\left[\vec{r}_{2}(t)-\vec{r}_{1}(t)\right]^3}\left[\vec{r}_{2}(t)-\vec{r}_{1}(t)\right]-\frac{Gm_3}{\left[\vec{r}_{3}(t)-\vec{r}_{1}(t)\right]^3}\left[\vec{r}_{3}(t)-\vec{r}_{1}(t)\right],
$$

$$
\dot{\vec{r}}_1(t)=\vec{v}_1.
$$

EoM в этой задаче представляет собой набор ОДУ. Задав два начальных условия, например начальную скорость и положение, мы можем свести задачу к двум ОДУ, где следующий шаг первого ОДУ зависит от предыдущего шага второго ОДУ. Правая сторона (RHS) EoM может быть описана следующей функцией:

In [ ]:
def RHS(t, y):
    """Calculate the RHS of the EoM, as described above.
    Parameters:
        y: array. Vector of length 12 holding the current position and velocity of the three objects 
           in the following manner: y = [x1, y2, x2, y2, x3, y3, vx1, vy1, vx2, vy2, vx3, vy3].
    Returns:
        z: array. Vector of length 12 holding the derivative of the current position and velocity 
           (the velocity and acceleration) of the three object in the following manner:
           z = [vx1, vy1, vx2, vy2, vx3, vy3, ax1, ay1, ax2, ay2, ax3, ay3].
    """
    
    # Allocate a vector to hold the output values
    z = np.zeros(12)
    # Define initial velocities and distances between objects
    z[:6] = [y[6], y[7], y[8], y[9], y[10], y[11]]
    r21 = ((y[2] - y[0])**2.0 + (y[3] - y[1])**2.0)**0.5
    r31 = ((y[4] - y[0])**2.0 + (y[5] - y[1])**2.0)**0.5
    r32 = ((y[4] - y[2])**2.0 + (y[5] - y[3])**2.0)**0.5
    # Pairwise forces
    Fx21 = G*m2*m1*(y[2] - y[0])/r21**3.0
    Fy21 = G*m2*m1*(y[3] - y[1])/r21**3.0
    Fx31 = G*m3*m1*(y[4] - y[0])/r31**3.0
    Fy31 = G*m3*m1*(y[5] - y[1])/r31**3.0
    Fx32 = G*m3*m2*(y[4] - y[2])/r32**3.0
    Fy32 = G*m3*m2*(y[5] - y[3])/r32**3.0
    # Accelerations
    z[6] = (Fx21 + Fx31)/m1
    z[7] = (Fy21 + Fy31)/m1
    z[8] = (-Fx21 + Fx32)/m2
    z[9] = (-Fy21 + Fy32)/m2
    z[10] = (-Fx31 - Fx32)/m3
    z[11] = (-Fy31 - Fy32)/m3
    
    return z

### Сохранение энергии и углового момента

Поскольку на систему не действуют никакие силы или крутящие моменты (и нет никакой формы потерь внутри системы), как энергия, так и угловой момент остаются постоянными. Это дает возможность проверить, достаточно ли точны наши расчеты. Кинетическая энергия, $K$, задается

$$
K=\frac{1}{2}m\vec v^2,
$$

где $m$ - масса, а $v$ - скорость. Потенциальная энергия $U$ между двумя объектами задается

$$
U=-\frac{Gm_1m_2}{r_{12}},
$$

где $G$ - гравитационная постоянная, $m_i$ - масса объекта $i$ и $r$ - расстояние между двумя объектами. Таким образом, энергии системы могут быть рассчитаны с помощью:

In [ ]:
def energy(z):
    """Calculate the mechanical energies of the three body system.
    Parameters:
        z: array. Vector of length 12 holding the current position and velocity of the three objects in the
           following manner: z = [x1, y2, x2, y2, x3, y3, vx1, vy1, vx2, vy2, vx3, vy3].
    Returns:
        U: float. The potential energies of the system.
        K: float. The kinetic energies of the system.
    """
    
    # Pairwise distance between objects
    r21 = ((z[2] - z[0])**2.0 + (z[3] - z[1])**2.0)**0.5
    r31 = ((z[4] - z[0])**2.0 + (z[5] - z[1])**2.0)**0.5
    r32 = ((z[4] - z[2])**2.0 + (z[5] - z[3])**2.0)**0.5
    
    # Calculate potential energies
    # First object is "free", second object is moved from infinity to a distance r21 from the first object.
    # Third object is affected gravitationally by both objects.
    U1 = 0
    U2 = -G*m1*m2/r21
    U3 = -G*m1*m3/r31 - G*m3*m2/r32
    U = U1 + U2 + U3
    
    # Calculate kinetic energies of the three objects
    K1 = 0.5*m1*(z[6]**2 + z[7]**2)
    K2 = 0.5*m2*(z[8]**2 + z[9]**2)
    K3 = 0.5*m3*(z[10]**2 + z[11]**2)
    K = K1 + K2 + K3
    
    return U, K

Кроме того, в этой модели абсолютный угловой момент $\vec L = \vec R\times m\vec v$ остается постоянным и может быть вычислен по:

In [ ]:
def angularMomentum(y):
    """Calculate absolute angular momentum of the three body system.
    Parameters:
        y:          array. Vector of length 12 holding the current position and velocity of the three objects
                    in the following manner: y = [x1, y2, x2, y2, x3, y3, vx1, vy1, vx2, vy2, vx3, vy3].
    Returns:
        L1, L2, L3: array. Total absolute angular momentum of the system.
    """
    
    L1 = m1*(y[0]*y[7] - y[1]*y[6])
    L2 = m2*(y[2]*y[9] - y[3]*y[8])
    L3 = m3*(y[4]*y[11] - y[5]*y[10])
    
    return [L1, L2, L3]

### Встроенная пара Рунге-Кутта

Теперь мы реализуем адаптивный метод Рунге-Кутты для решения EoM. Короче говоря, это схема, которая использует два разных метода Рунге-Кутты разного порядка для получения оценки локальной ошибки усечения. Таким образом, можно более или менее решить, какую точность мы хотим, регулируя размер шага для каждой итерации. Еще одно преимущество метода адаптивного размера шага заключается в том, что мы достигаем большей точности там, где это необходимо, и меньшей точности там, где она не критична. Однако эти методы могут стать более требовательными к вычислениям и, следовательно, не улучшат нашу реализацию в каждом конкретном случае.

Реализация также может быть выполнена с помощью метода с постоянным размера шага, который описан в разделе анимации ниже.

Мы собираемся использовать встроенную пару Рунге-Кутта-Фельберг порядка 4 / порядка 5 (RKF45), как описано в [Adaptive Runge-Kutta Methods](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/adaptive_runge_kutta_methods.ipynb):

In [ ]:
def ode45(f,t,y,h):
    """Calculate next step of an initial value problem (IVP) of an ODE with a RHS described
    by the RHS function with an order 4 approx. and an order 5 approx.
    Parameters:
        t: float. Current time.
        y: float. Current step (position).
        h: float. Step-length.
    Returns:
        q: float. Order 2 approx.
        w: float. Order 3 approx.
    """
    
    s1 = f(t, y)
    s2 = f(t + h/4.0, y + h*s1/4.0)
    s3 = f(t + 3.0*h/8.0, y + 3.0*h*s1/32.0 + 9.0*h*s2/32.0)
    s4 = f(t + 12.0*h/13.0, y + 1932.0*h*s1/2197.0 - 7200.0*h*s2/2197.0 + 7296.0*h*s3/2197.0)
    s5 = f(t + h, y + 439.0*h*s1/216.0 - 8.0*h*s2 + 3680.0*h*s3/513.0 - 845.0*h*s4/4104.0)
    s6 = f(t + h/2.0, y - 8.0*h*s1/27.0 + 2*h*s2 - 3544.0*h*s3/2565 + 1859.0*h*s4/4104.0 - 11.0*h*s5/40.0)
    w = y + h*(25.0*s1/216.0 + 1408.0*s3/2565.0 + 2197.0*s4/4104.0 - s5/5.0)
    q = y + h*(16.0*s1/135.0 + 6656.0*s3/12825.0 + 28561.0*s4/56430.0 - 9.0*s5/50.0 + 2.0*s6/55.0)
    
    return w, q

Чтобы получить некоторую основу для сравнения, мы также будем использовать обычный (четвертого порядка) метод Рунге-Кутты, как описано в [Метод Рунге-Кутты](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/runge_kutta_method.ipynb) (этот метод будет использоваться для создания анимации):

In [ ]:
def rk4step(f, t, y, h):
    """Calculate next step of an IVP of an ODE with a RHS described by the RHS function with RK4.
    Parameters:
        f: function. Right hand side of the ODE.
        t: float. Current time.
        y: float. Current step (position).
        h: float. Step-length.
    Returns:
        q: float. Order 2 approx.
        w: float. Order 3 approx.
    """
    
    s1 = f(t, y)
    s2 = f(t + h/2.0, y + h*s1/2.0)
    s3 = f(t + h/2.0, y + h*s2/2.0)
    s4 = f(t + h, y + h*s3)
    
    return y + h/6.0*(s1 + 2.0*s2 + 2.0*s3 + s4)

### Вычисления

Время собрать все это воедино и наметить результаты. Первоначально это решение описывает проблему с тремя телами, но, при выборе одной из масс, равной нулю, система будет вести себя так, как если бы это была система из двух тел.

Для начала мы выберем начальные условия и установим глобальные константы:

In [ ]:
# Set gravitaional constant and masses.
# The gravitaional constant is set to 1 for simplicity.
G = 1.
m1 = 1.
m2 = 1.
m3 = 1.

# Period of calculations
T = 5
# Tolerance
TOL = 0.00001
# Maximum number of steps
maxi = 1000
# "Protector-constant" for small w
theta = 0.001
# Number of steps (RK4)
n = 1000

# Different initial conditions to try out, on the form
# y = [x1, y2, x2, y2, x3, y3, vx1, vy1, vx2, vy2, vx3, vy3]
# z0 = [2., 2., 0., 0., -2., -2., 0.2, -0.2, 0., 0., -0.2, 0.2]
# z0 = [-0.970, 0.243, 0.970, -0.243, 0., 0., -0.466, -0.433, -0.466, -0.433, 2*0.466, 2*0.433]
# z0 = [2., 0., -2., 0., 0., 0., 0., -0.6, -0.6, 0., 0., 0.]
z0 = [1., 0.000001, -1., 0., 0., 0., 0., -0.4, 0., 0.4, 0., 0.]

Теперь мы готовы выполнить расчеты и построить график результатов. Чтобы разобраться, вы должны запустить блокнот самостоятельно и попробовать различные начальные условия $z_0$. Например, для последних $z_{0}$, перечисленных выше, если вы установите $y_{2} = 0$ вместо, скажем, $0.000001$, орбиты стабильны. Как мы увидим ниже, система трех тел очень чувствительна к начальным условиям.

In [ ]:
# First step and initial time
h = T/n
t = 0.

# Allocate matrices and fill with initial conditions
Z = np.zeros((12, maxi + 1))
Z[:, 0] = z0
E = np.zeros((maxi + 1, 2))
E[0, :] = energy(z0)
L = np.zeros((maxi + 1, 3))
L[0, :] = angularMomentum(z0)

# Declare iteration integer
i = 0
e=1.
# Perform ode45 calculations
tic = time.time()
while (t < T) & (i < maxi):
    w, q = ode45(RHS, t, Z[:, i], h)
    e = max(abs((w - q)/np.maximum(w, theta)))
    if e > TOL:
        h = 0.95*(TOL*e)**(1/5)*h 
        w, q = ode45(RHS, t, q, h)
        e = max(abs((w - q)/np.maximum(w, theta)))
        while e > TOL:
            h = h/2.
            w, q = ode45(RHS, t, q, h)
            e = max(abs((w - q)/maximum(w, theta)))
    if e < 0.1*TOL:
        h = h*2.
    Z[:, i+1] = q
    E[i+1, :] = energy(Z[:, i+1])
    L[i+1, :] = angularMomentum(Z[:, i+1])
    t += h
    i += 1

print("%.5f s, run time of adaptive RK45 method." % (time.time() - tic))
    
# Print number of steps used
if (i == maxi):
    print('%i, maximum number of steps reached by adaptive RK45 method.' % i)
else:
    print('%i, steps used by adaptive RK45 method.' % i)

In [ ]:
# Plot result
plt.figure()

# Position
plt.subplot(2, 2, (1, 3))
plt.plot(Z[0, 0:i], Z[1, 0:i], '-r', label='m1')
plt.plot(Z[2, 0:i], Z[3, 0:i], '-b', label='m2')
plt.plot(Z[4, 0:i], Z[5, 0:i], '-g', label='m3')
plt.plot(Z[0, 0], Z[1, 0], '*r', label='Initial position m1')
plt.plot(Z[2, 0], Z[3, 0], '*b', label='Initial position m2')
plt.plot(Z[4, 0], Z[5, 0], '*g', label='Initial position m3')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='best')
plt.title('Trajectories')

# Mechanical energy
plt.subplot(2, 2, 2)
plt.plot(E[0:i, 0], 'g', label='K')
plt.plot(E[0:i, 1], 'b', label='U')
plt.plot(E[0:i, 0] + E[0:i ,1], 'm', label='Total mechanical energy E')
plt.ylabel('E')
plt.legend(loc='best')
plt.title('Mechanical energy and absolute angular momentum')

# Angular momentum
plt.subplot(2, 2, 4)
plt.plot(L[0:i, 0], 'r', label='m1')
plt.plot(L[0:i, 1], 'b', label='m2')
plt.plot(L[0:i, 2], 'g', label='m3')
plt.plot(L[0:i, 0] + L[0:i, 1] + L[0:i, 2], 'm', label='Total L')
plt.xlabel('Steps, i')
plt.ylabel('L')
plt.legend(loc=3)
plt.show()

При начальных условиях $z_0$, выбранных выше, система из трех тел нестабильна, поэтому $m_2$ дрейфует сама по себе, в то время как $m_1$ и $m_3$ участвуют в плотном танце вместе. Кроме того, общий угловой момент $L$ и общая механическая энергия $E$ достаточно постоянны на протяжении всего расчета, что указывает на достаточно точные расчеты.

Этот метод часто требует нескольких шагов. Как следствие, сюжет может показаться несколько плохим, хотя точность может быть высокой. Используя метод RK4 без адаптации размера шага, мы получаем другую картину.

In [ ]:
# Set constant step size
h = T/n

# Set initial time
t = 0.

# Allocate matrices and fill with initial conditions
Z2 = np.zeros((12, n+1))
Z2[:, 0] = z0
E2 = np.zeros((n+1, 2))
E2[0, :] = energy(z0)
L2 = np.zeros((n+1, 3))
L2[0, :] = angularMomentum(z0)

tic = time.time()
for i in range(0, n):
    Z2[:, i+1] = rk4step(RHS, t, Z2[:, i], h)
    E2[i+1, :] = energy(Z2[:, i+1])
    L2[i+1, :] = angularMomentum(Z2[:, i+1])
    t += h


print("%.5f s, run time of RK4 method, with %i steps."% (time.time() - tic, n))

In [ ]:
# Plot result
plt.figure()

# Position
plt.subplot(2, 2, (1, 3))
plt.plot(Z2[0, 0:i], Z2[1, 0:i], '-r', label='m1')
plt.plot(Z2[2, 0:i], Z2[3, 0:i], '-b', label='m2')
plt.plot(Z2[4, 0:i], Z2[5, 0:i], '-g', label='m3')
plt.plot(Z2[0, 0], Z2[1, 0], '*r', label='Initial position m1')
plt.plot(Z2[2, 0], Z2[3, 0], '*b', label='Initial position m2')
plt.plot(Z2[4, 0], Z2[5, 0], '*g', label='Initial position m3')
plt.xlabel('x')
plt.ylabel('y')
plt.legend(loc='best')
plt.title('Trajectories')

# Mechanical energy
plt.subplot(2, 2, 2)
plt.plot(E2[0:i, 0], 'g', label='K')
plt.plot(E2[0:i, 1], 'b', label='U')
plt.plot(E2[0:i, 0] + E2[0:i, 1], 'm', label='Total mechanical energy, E')
plt.ylabel('E')
plt.legend(loc='best')
plt.title('Mechanical energy and absolute angular momentum.')

# Angular momentum
plt.subplot(2, 2, 4)
plt.plot(L2[0:i, 0], 'r', label='m1')
plt.plot(L2[0:i, 1], 'b', label='m2')
plt.plot(L2[0:i, 2], 'g', label='m3')
plt.plot(L2[0:i, 0] + L2[0:i, 1] + L2[0:i, 2], 'm', label='Total L')
plt.xlabel('Steps, i')
plt.ylabel('L')
plt.legend(loc='best')
plt.show()

Из одного взгляда на общую механическую энергию $E$ и общий угловой момент $L$ очевидно, что этот расчет ошибочен. Мы не можем доверять траекториям, полученным обычным методом Рунге-Кутты четвертого порядка (RK4) с разбиением в 1000 шагов. Для сравнения, адаптивный метод Рунге-Кутты (RK45) показал лучшие результаты, используя всего 224 шага, хотя он примерно удвоил время выполнения. Теперь, если мы используем начальное число шагов $n$ = 10 000, обычный метод RK4 дает правдоподобные траектории (попробуйте сами). Однако адаптивный метод RK45 дает те же результаты примерно за то же количество шагов, что и раньше, что позволяет сократить время его выполнения.

### Создание анимации

Самый простой способ создать анимацию - использовать функцию постоянного размера шага, такую как метод Рунге-Кутты четвертого порядка (последнее вычисление будет анимировано). Теперь мы собираемся анимировать предыдущие вычисления с помощью $n$ = 10 000. Анимация во введении была сделана со следующим кодом:

In [ ]:
# Set up the figure
fig = plt.figure()
ax = plt.axes(xlim=(np.min(Z2[[0,2,4],:]), np.max(Z2[[0,2,4],:])), 
              ylim=(np.min(Z2[[1,3,5],:]), np.max(Z2[[1,3,5],:])))
ax.set_aspect('equal')

# Define the different elements in the animation
tail1, = ax.plot([],[],'r') # Tail obj. 1
tail2, = ax.plot([],[],'b') # Tail obj. 2
tail3, = ax.plot([],[],'g') # Tail obj. 3
obj1, = ax.plot([],[],'ro') # Obj. 1
obj2, = ax.plot([],[],'bo') # Obj. 2
obj3, = ax.plot([],[],'go') # Obj. 3

# Calculates the number of frames
FPS = 15
framesNum = int(FPS*T)

# Animation function. This is called sequentially.
def animate(j):
    i = j*int(n/framesNum)
    obj1.set_data(Z2[0,i],Z2[1,i])
    obj2.set_data(Z2[2,i],Z2[3,i])
    obj3.set_data(Z2[4,i],Z2[5,i])
    tail1.set_data(Z2[0,0:i],Z2[1,0:i])
    tail2.set_data(Z2[2,0:i],Z2[3,0:i])
    tail3.set_data(Z2[4,0:i],Z2[5,0:i])

# Create animation
anim = animation.FuncAnimation(fig, animate, frames=framesNum)

# Save animation
# If this don't work for you, try using the another writer (ffmpeg, mencoder, imagemagick),
# or another file extension (.mp4, .gif, .ogg, .ogv, .avi etc.). Make sure that you have
# the codec and the writer installed on your system.
anim.save('ThreeBodyProblem.gif', writer='imagemagick', fps=FPS)

# Close plot
plt.close(anim._fig)

# Display the animation
with open('ThreeBodyProblem.gif','rb') as file:
    display(Image(file.read()))

### References and further reading

[1] Sauer, T.: *Numerical Analysis international edition*, 2nd edition, Pearson 2014

[2] *Runge-Kutta methods*, Wikipedia, https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods (Acquired: 13.03.16)

[3] Montgomery, R.: *A New Solution to the Three-Body Problem*, http://www.ams.org/notices/200105/fea-montgomery.pdf, 2001

---

### Упражнения

1) Попробуйте использовать начальные условия z0 = [2.0,2.0,0.0,0.0,-2.0,-2.0,0.2,-0.2,0.0,0.0,-0.2,0.2]. Стабильна ли эта система? Что произойдет, если вы измените один из начальных параметров на небольшую величину (например, x_3 = 0.0001)? Можете ли вы объяснить, что происходит?

2) Решите задачу с двумя телами, изменив код.

3) Создайте несколько примеров, подтверждающих законы Кеплера.